In [1]:
import numpy as np
import matplotlib.pyplot as plt
from elf.io import open_file
import z5py
from pathlib import Path

import os
import imageio
import mobie
import mobie.metadata as metadata

from cryofib.n5_utils import read_volume, tif2n5

# Set paths

In [2]:
scratch_dir = Path("/scratch/buglakova/data/cryofib/registration_fluo/F107_A2_3dclem")
fluo_dir = scratch_dir / "fluo"
em_dir = scratch_dir / "em"
segm_em_nuclei_dir = scratch_dir / "segm_em_nuclei"

## Set information about mobie project

In [3]:
mobie_project_folder = "/scratch/buglakova/data/cryofib/registration_fluo/mobie_projects/fibsem-registration-affine/data"

## Create separate datasets

In [4]:
dataset_name = "registration"
dataset_folder = os.path.join(mobie_project_folder, dataset_name)

# the platform and number of jobs used for computation.
# choose 'local' to run computations on your machine.
# for large data, it is also possible to run computation on a cluster;
# for this purpose 'slurm' (for slurm cluster) and 'lsf' (for lsf cluster) are currently supported
target = "local"
max_jobs = 4

# The 'default' image for our example dataset is a 2d EM slice showing an overview of the dataset.
input_file = str(fluo_dir / "fluo.n5")
input_key = "raw"
# This is the name that will be given to the image source in mobie.
raw_name = "fluo"
# The name of the menu from which the image can be added to the viewer.
# Here, we choose "em", because this is an EM image slice.
menu_name = "fluo"

# We need some metadata to create the n5-file in big-data-viewer format:
# - unit: the phyiscal unit of the coordinate system
# - resolution: the size of one voxel in the physical unit, this needs to be a tuple/list of length 3,
#               specifying the size for each of the 3 spatial dimensions
# - chunks: the size of the chunks (in voxels) that are used to store the output file.
#           good choices are usually (1, 512, 512) for 2d data and (64, 64, 64) for 3d data
# - scale_factors: the scale factors used for downsampling the input when creating the image pyramid
#                  this needs to be a list, where each entry specifies the scale factors for the 3 axes.
# Note that axes are always listed in the order ZYX here (in the java implementation of mobie / big-data-viewer the axis convention is XYZ).
# Also note that the values for all three axes (ZYX) need to be specified. In the case of 2d data, the value
# for Z should be set to 1.

f_out = z5py.File(input_file, "r")
ds = f_out[input_key]

unit = ds.attrs["unit"]
resolution = ds.attrs["resolution"]
chunks = (1, 512, 512)
scale_factors = 4 * [[1, 2, 2]]

mobie.add_image(
    input_path=input_file, 
    input_key=input_key,  # the input is a single tif image, so we leave input_key blank
    root=mobie_project_folder,
    dataset_name=dataset_name,
    image_name=raw_name,
    menu_name=menu_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    is_default_dataset=True,  # mark this dataset as the default dataset that will be loaded by mobie
    target=target,
    max_jobs=max_jobs,
    unit=unit
)

/home/buglakov/miniconda3/envs/mobie/lib/python3.10/site-packages/luigi/parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_registration_fluo, max_jobs=4, config_dir=tmp_registration_fluo/configs, target=local, dependency=DummyTask, input_path=/scratch/buglakova/data/cryofib/registration_fluo/F107_A2_3dclem/fluo/fluo.n5, input_key=raw, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], dtype=None, int_to_uint=False, metadata_format=bdv.n5, metadata_dict={"resolution": [1849.5, 221.4, 221.4], "unit": "nm", "setup_name": "fluo"}, output_path=/scratch/buglakova/data/cryofib/registration_fluo/mobie_projects/fibsem-registration-affine/data/registration/images/bdv-n5/fluo.n5, output_key_prefix=, force_copy=False, skip_existing_levels=Fals

ValueError: /scratch/buglakova/data/cryofib/registration_fluo/mobie_projects/fibsem-registration-affine/data/registration/images/bdv-n5/fluo.xml does not exist.

In [ ]:
mobie_project_folder = "/scratch/buglakova/data/cryofib/registration_fluo/mobie_projects/fibsem-registration/data"

dataset_name = "registration"
dataset_folder = os.path.join(mobie_project_folder, dataset_name)
target = "local"
max_jobs = 4

input_file = str(em_dir / "em.n5")
input_key = "raw"
raw_name = "em_registered_rotate_scale"
menu_name = "em_registered_rotate_scale"

f_out = z5py.File(input_file, "r")
ds = f_out[input_key]

unit = ds.attrs["unit"]
resolution = ds.attrs["resolution"]
chunks = (1, 512, 512)
scale_factors = 4 * [[2, 2, 2]]

affine_transform = [ 2.99703223e+01, -3.87356614e-01,  1.30206802e+00, -1.53353979e+04,
 -1.33408452e+00, -8.70199929e+00,  2.92510688e+01,  1.21702925e+05,
 -0.00000000e+00, -3.90400456e+01, -6.53296224e+00,  1.37783569e+05]


mobie.add_image(
    input_path=input_file, 
    input_key=input_key,  # the input is a single tif image, so we leave input_key blank
    root=mobie_project_folder,
    dataset_name=dataset_name,
    image_name=raw_name,
    menu_name=menu_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    is_default_dataset=True,  # mark this dataset as the default dataset that will be loaded by mobie
    target=target,
    max_jobs=max_jobs,
    unit=unit,
    transformation=affine_transform
)

In [ ]:
dataset_name = "registration"
dataset_folder = os.path.join(mobie_project_folder, dataset_name)
target = "local"
max_jobs = 4

input_file = str(segm_em_nuclei_dir / "em_nuclei.n5")
input_key = "raw"
raw_name = "em_segm_nuclei"
menu_name = "em_segm_nuclei"

f_out = z5py.File(input_file, "r")
ds = f_out[input_key]

unit = ds.attrs["unit"]
resolution = ds.attrs["resolution"]
chunks = (1, 512, 512)
scale_factors = 4 * [[2, 2, 2]]

mobie.add_segmentation(
    input_path=input_file, 
    input_key=input_key,  # the input is a single tif image, so we leave input_key blank
    root=mobie_project_folder,
    dataset_name=dataset_name,
    segmentation_name=raw_name,
    menu_name=menu_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    is_default_dataset=True,  # mark this dataset as the default dataset that will be loaded by mobie
    target=target,
    max_jobs=max_jobs,
    unit=unit
)